In [2]:
pip install BeautifulSoup4

     |████████████████████████████████| 112kB 6.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install geopy

     |████████████████████████████████| 112kB 5.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install geocoder

     |████████████████████████████████| 102kB 12.1MB/s ta 0:00:01
     |████████████████████████████████| 92kB 4.8MB/s  eta 0:00:01
     |████████████████████████████████| 829kB 10.0MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [5]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [6]:
data = requests.get("https://de.wikipedia.org/wiki/Kategorie:Stadtteil_von_Kassel").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})
kl_df.head()

,Neighborhood
0,Bad Wilhelmshöhe
1,Bettenhausen (Kassel)
2,Brasselsberg (Kassel)
3,Fasanenhof (Kassel)
4,Forstfeld (Kassel)


In [7]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kassel, Germany'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [8]:
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [9]:
coords

[[51.31660206723354, 9.420743286192975],
 [51.313760923175366, 9.507323713649857],
 [51.288650000000075, 9.415340000000072],
 [51.33063008050087, 9.519509995528002],
 [51.29713299480513, 9.537739072728195],
 [51.36275042584529, 9.432810105233953],
 [51.34066000000007, 9.466560000000072],
 [51.31696952607233, 9.441507816915529],
 [51.35951194980332, 9.55925750393354],
 [51.31602650255086, 9.501225751639833],
 [51.28327000000007, 9.467520000000036],
 [51.31853000000007, 9.511830000000032],
 [51.28043000000008, 9.431680000000028],
 [49.24248000000006, 8.26484000000005],
 [51.277600000000064, 9.452090000000055],
 [51.34541000000007, 9.496480000000076],
 [51.34808250000002, 9.46928250000001],
 [51.31853000000007, 9.511830000000032],
 [51.29676000000006, 9.451780000000042],
 [51.31470000000007, 9.511140000000069],
 [51.624627050284055, 9.617859832926344],
 [51.32021142771955, 9.44433656148823],
 [51.28606158954837, 9.496284983849655],
 [51.31232356263808, 9.465765777081616],
 [51.31580810883

In [10]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']
kl_df

,Neighborhood,Latitude,Longitude
0,Bad Wilhelmshöhe,51.316602,9.420743
1,Bettenhausen (Kassel),51.313761,9.507324
2,Brasselsberg (Kassel),51.288650,9.415340
3,Fasanenhof (Kassel),51.330630,9.519510
4,Forstfeld (Kassel),51.297133,9.537739
5,Harleshausen,51.362750,9.432810
6,Jungfernkopf,51.340660,9.466560
7,Kirchditmold,51.316970,9.441508
8,Kragenhof,51.359512,9.559258
9,Mitte (Kassel),51.316027,9.501226


In [11]:
kl_df.to_csv("kl_df.csv", index=False)

In [12]:
address = 'Kassel, Germany'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [13]:
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [14]:
# define Foursquare Credentials and Version
CLIENT_ID = 'U4HX1OECYTILUXOM2AQKKOR34EL3TFOROT5ZWOJR0HTLSB2S' # your Foursquare ID
CLIENT_SECRET = 'QX3HHS1QCI1WKCTPMENDFO154JQGADGF4LCHMQUFUKAHOKRD' # your Foursquare Secret
VERSION = '20200605' # Foursquare API version

In [15]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [16]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1075, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Bad Wilhelmshöhe,51.316602,9.420743,Bergpark Wilhelmshöhe,51.314641,9.412708,Park
1,Bad Wilhelmshöhe,51.316602,9.420743,Schloss Wilhelmshöhe,51.315103,9.416313,Palace
2,Bad Wilhelmshöhe,51.316602,9.420743,Kurhessen Therme,51.313561,9.426625,Hot Spring
3,Bad Wilhelmshöhe,51.316602,9.420743,Gutshof,51.313556,9.429114,German Restaurant
4,Bad Wilhelmshöhe,51.316602,9.420743,Brauhaus zum Rammelsberg,51.315868,9.436689,Brewery


In [17]:
venues_df['VenueCategory'].unique()[:50]

array(['Park', 'Palace', 'Hot Spring', 'German Restaurant', 'Brewery',
       'Greek Restaurant', 'Fountain', 'Monument / Landmark',
       'Indian Restaurant', 'Castle', 'Hotel', 'Historic Site', 'Café',
       'Ice Cream Shop', 'American Restaurant', 'Steakhouse',
       'Restaurant', 'Drugstore', 'Chinese Restaurant', 'Tram Station',
       'Food', 'Coffee Shop', 'Bakery', 'College Gym', 'Bookstore',
       'Trattoria/Osteria', 'Platform', 'Bank', 'Supermarket',
       'Art Museum', 'Art Gallery', 'Farmers Market',
       'African Restaurant', 'Music Venue', 'Italian Restaurant',
       'Nightclub', 'Plaza', 'Bagel Shop', 'Theater',
       'Gym / Fitness Center', 'History Museum', 'Cocktail Bar',
       'Science Museum', 'Pool', 'Multiplex', 'Clothing Store',
       'Automotive Shop', 'School', 'Scenic Lookout', 'Pub'], dtype=object)

In [18]:

# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(1075, 116)


,Neighborhoods,African Restaurant,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Garden,Big Box Store,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Business Service,Café,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Department Store,Dessert Shop,Doner Restaurant,Drugstore,Electronics Store,Event Service,Event Space,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food Truck,Fountain,Furniture / Home Store,Garden Center,Gas Station,Gastropub,German Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Historic Site,History Museum,Hookah Bar,Hot Spring,Hotel,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Liquor Store,Locksmith,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Palace,Park,Perfume Shop,Pet Store,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Rest Area,Restaurant,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Soccer Field,Soccer Stadium,Spanish Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Theater,Toy / Game Store,Trail,Train Station,Tram Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vietnamese Restaurant,Wine Shop
0,Bad Wilhelmshöhe,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bad Wilhelmshöhe,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Bad Wilhelmshöhe,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bad Wilhelmshöhe,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bad Wilhelmshöhe,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(26, 116)


,Neighborhoods,African Restaurant,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beer Garden,Big Box Store,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Bus Stop,Business Service,Café,Castle,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Concert Hall,Construction & Landscaping,Department Store,Dessert Shop,Doner Restaurant,Drugstore,Electronics Store,Event Service,Event Space,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food Truck,Fountain,Furniture / Home Store,Garden Center,Gas Station,Gastropub,German Restaurant,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Historic Site,History Museum,Hookah Bar,Hot Spring,Hotel,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Italian Restaurant,Jewelry Store,Juice Bar,Lake,Light Rail Station,Liquor Store,Locksmith,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Palace,Park,Perfume Shop,Pet Store,Pharmacy,Pizza Place,Platform,Plaza,Pool,Pub,Rest Area,Restaurant,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Soccer Field,Soccer Stadium,Spanish Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Theater,Toy / Game Store,Trail,Train Station,Tram Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Vietnamese Restaurant,Wine Shop
0,Bad Wilhelmshöhe,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.055556,0.027778,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.055556,0.027778,0.027778,0.000000,0.000000,0.027778,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.027778,0.000000,0.00,0.000000,0.000000,0.055556,0.027778,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.027778,0.083333,0.027778,0.027778,0.000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.027778,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.027778,0.000,0.000000,0.000000,0.000000
1,Bettenhausen (Kassel),0.010101,0.010101,0.030303,0.040404,0.010101,0.010101,0.00,0.010101,0.020202,0.010101,0.030303,0.010101,0.010101,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050505,0.000000,0.000000,0.030303,0.040404,0.050505,0.000000,0.000000,0.000000,0.000000,0.000000,0.020202,0.040404,0.000000,0.000000,0.000000,0.010101,0.000000,0.000000,0.010101,0.000000,0.000000,0.000000,0.00,0.010101,0.000000,0.020202,0.000000,0.000000,0.000000,0.030303,0.010101,0.010101,0.010101,0.010101,0.000000,0.030303,0.010101,0.000000,0.000,0.000000,0.030303,0.000,0.000000,0.010101,0.010101,0.000000,0.010101,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.010101,0.010101,0.010101,0.010101,0.020202,0.000000,0.010101,0.010101,0.010101,0.000000,0.000000,0.000000,0.020202,0.010101,0.020202,0.000000,0.000000,0.010101,0.010101,0.020202,0.000000,0.000000,0.010101,0.010101,0.000000,0.000000,0.000000,0.010101,0.000000,0.060606,0.000000,0.010101,0.000000,0.000000,0.000000,0.000000,0.020202,0.000,0.030303,0.010101,0.000000
2,Brasselsberg (Kassel),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0

In [20]:
len(kl_grouped[kl_grouped["Coffee Shop"] > 0])

9

In [21]:
kl_grouped['Café'] > 0
kl_grouped["Coffee Shop"] > 0
kl_cafe = kl_grouped[["Neighborhoods","Café","Coffee Shop"]]
kl_cafe.head()

,Neighborhoods,Café,Coffee Shop
0,Bad Wilhelmshöhe,0.055556,0.027778
1,Bettenhausen (Kassel),0.050505,0.050505
2,Brasselsberg (Kassel),0.000000,0.000000
3,Fasanenhof (Kassel),0.052632,0.026316
4,Forstfeld (Kassel),0.000000,0.000000


In [22]:

# set number of clusters
kclusters = 4

kl_clustering = kl_cafe.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 1, 0, 0, 3, 2, 0, 1], dtype=int32)

In [23]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_cafe.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [24]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Café,Coffee Shop,Cluster Labels
0,Bad Wilhelmshöhe,0.055556,0.027778,1
1,Bettenhausen (Kassel),0.050505,0.050505,1
2,Brasselsberg (Kassel),0.000000,0.000000,0
3,Fasanenhof (Kassel),0.052632,0.026316,1
4,Forstfeld (Kassel),0.000000,0.000000,0


In [25]:
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(26, 6)


,Neighborhood,Café,Coffee Shop,Cluster Labels,Latitude,Longitude
0,Bad Wilhelmshöhe,0.055556,0.027778,1,51.316602,9.420743
1,Bettenhausen (Kassel),0.050505,0.050505,1,51.313761,9.507324
2,Brasselsberg (Kassel),0.000000,0.000000,0,51.288650,9.415340
3,Fasanenhof (Kassel),0.052632,0.026316,1,51.330630,9.519510
4,Forstfeld (Kassel),0.000000,0.000000,0,51.297133,9.537739


In [26]:

# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(26, 6)


,Neighborhood,Café,Coffee Shop,Cluster Labels,Latitude,Longitude
12,Nordshausen,0.000000,0.000000,0,51.280430,9.431680
22,Waldau (Kassel),0.000000,0.000000,0,51.286062,9.496285
20,Vorderer Westen,0.000000,0.000000,0,51.624627,9.617860
16,Rothenditmold,0.000000,0.000000,0,51.348083,9.469283
15,Philippinenhof-Warteberg,0.000000,0.000000,0,51.345410,9.496480
14,Oberzwehren,0.000000,0.000000,0,51.277600,9.452090
13,Oberneustadt,0.000000,0.000000,0,49.242480,8.264840
10,Niederzwehren,0.000000,0.000000,0,51.283270,9.467520
25,Wolfsanger / Hasenhecke,0.000000,0.000000,0,51.343815,9.542803
2,Brasselsberg (Kassel),0.000000,0.000000,0,51.288650,9.415340


In [27]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Café,Coffee Shop,Cluster Labels,Latitude,Longitude
12,Nordshausen,0.0,0.0,0,51.280430,9.431680
22,Waldau (Kassel),0.0,0.0,0,51.286062,9.496285
20,Vorderer Westen,0.0,0.0,0,51.624627,9.617860
18,Süsterfeld-Helleböhn,0.0,0.0,0,51.296760,9.451780
16,Rothenditmold,0.0,0.0,0,51.348083,9.469283
15,Philippinenhof-Warteberg,0.0,0.0,0,51.345410,9.496480
14,Oberzwehren,0.0,0.0,0,51.277600,9.452090
13,Oberneustadt,0.0,0.0,0,49.242480,8.264840
10,Niederzwehren,0.0,0.0,0,51.283270,9.467520
25,Wolfsanger / Hasenhecke,0.0,0.0,0,51.343815,9.542803


In [33]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Café,Coffee Shop,Cluster Labels,Latitude,Longitude
11,Nord-Holland (Kassel),0.06,0.04,1,51.318530,9.511830
24,Wesertor (Kassel),0.04,0.04,1,51.315808,9.505281
17,Südstadt (Kassel),0.06,0.04,1,51.318530,9.511830
19,Unterneustadt,0.04,0.04,1,51.314700,9.511140
1,Bettenhausen (Kassel),0.06,0.04,1,51.313761,9.507324
9,Mitte (Kassel),0.06,0.04,1,51.316027,9.501226


In [34]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Café,Coffee Shop,Cluster Labels,Latitude,Longitude
7,Kirchditmold,0.045455,0.000000,2,51.316970,9.441508
6,Jungfernkopf,0.052632,0.000000,2,51.340660,9.466560
3,Fasanenhof (Kassel),0.054054,0.000000,2,51.330630,9.519510
21,Wahlershausen,0.044444,0.000000,2,51.320211,9.444337
23,Wehlheiden,0.025974,0.012987,2,51.312324,9.465766


In [35]:
kl_merged.loc[kl_merged['Cluster Labels'] == 4]

,Neighborhood,Café,Coffee Shop,Cluster Labels,Latitude,Longitude
